# RecSys M 2021 - Exercise 3

The aims of this exercise are:
- Explore a different recommendation dataset
- Develop and evaluate baseline recommender systems
- Implement hybrid recommender models
- Explore diversification issues in recommender systems
- Revise other material from the lectures.

As usual, there is a corresponding Quiz on Moodle for this Exercise, which should be answered as you proceed. For more details, see the Exercise 3 specification.

# Part-Pre. Preparation

## Pre 1. Setup Block

This exercise will use the Goodreads dataset for books. These blocks setup the data files, Python etc.

In [1]:
!rm -rf ratings* books* to_read* test*

!curl -o ratings.csv http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-ratings.csv
!curl -o books.csv http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-books.csv
!curl -o to_read.csv http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-to_read.csv
!curl -o test.csv http://www.dcs.gla.ac.uk/~craigm/recsysH/coursework/final-test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7631k  100 7631k    0     0  4140k      0  0:00:01  0:00:01 --:--:-- 4138k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2366k  100 2366k    0     0  1823k      0  0:00:01  0:00:01 --:--:-- 1823k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7581k  100 7581k    0     0  4759k      0  0:00:01  0:00:01 --:--:-- 4756k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1895k  100 1895k    0     0  1234k      0  0:00:01  0:00:01 --:--:-- 1234k


We're going to use the Spotlight library in this exercise - see https://github.com/maciejkula/spotlight - and its documentation at https://maciejkula.github.io/spotlight/. You can install this direct from Git, but using Craig's patched version as done below.

In [2]:
!pip install git+https://github.com/cmacdonald/spotlight.git@master#egg=spotlight

  Cloning https://github.com/cmacdonald/spotlight.git (to revision master) to /tmp/pip-install-w0vtc1i4/spotlight_de28b3d494744e59bc813d2052a19aad
  Running command git clone -q https://github.com/cmacdonald/spotlight.git /tmp/pip-install-w0vtc1i4/spotlight_de28b3d494744e59bc813d2052a19aad


In [3]:
# Import modules.
from collections import defaultdict
from itertools import count
from typing import Sequence, Tuple

from scipy.stats import rankdata
from sklearn.preprocessing import minmax_scale
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import mrr_score, precision_recall_score
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.interactions import Interactions
import numpy as np
import pandas as pd
import torch
import torch.nn as nn


SEED = 20
BPRMF = None

## Pre 2. Data Preparation

Let's load the dataset into dataframes.

In [4]:
ratings_df = pd.read_csv('ratings.csv')
books_df = pd.read_csv('books.csv')
to_read_df = pd.read_csv('to_read.csv')
test = pd.read_csv('test.csv')

In [5]:
def str_col(df: pd.DataFrame) -> None:
  '''
  Stringify the ID columns.

  Parameters
  ----------
  df : the dataframe whose ID columns need stringifing.
  '''

  if 'user_id' in df.columns:
    df['user_id'] = 'u' + df.user_id.astype(str)

  if 'book_id' in df.columns:
    df['book_id'] = 'b' + df.book_id.astype(str)


# Cut down the number of items and users.
counts = ratings_df[ratings_df['book_id'] < 2000].groupby(['book_id']).count().reset_index()
valid_books = counts[counts['user_id'] >= 10][['book_id']]
books_df = books_df.merge(valid_books, on = 'book_id')
ratings_df = ratings_df[ratings_df['user_id'] < 2000].merge(valid_books, on = 'book_id')
to_read_df = to_read_df[to_read_df['user_id'] < 2000].merge(valid_books, on = 'book_id')
test = test[test['user_id'] < 2000].merge(valid_books, on = 'book_id')

str_col(books_df)
str_col(ratings_df)
str_col(to_read_df)
str_col(test)

In [6]:
books_df

,Unnamed: 0,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,0,b1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,b2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,2,b3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,3,b4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,4,b5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,1989,b1990,11331421,11331421,6504537,83,1590514637,9.781591e+12,"Jan-Philipp Sendker, Kevin Wiliarty",2002.0,Das Herzenhören,The Art of Hearing Heartbeats,eng,3.98,41647,50338,5650,704,2697,10330,19670,16937,https://images.gr-assets.com/books/1320437247m...,https://images.gr-assets.com/books/1320437247s...
1822,1990,b1991,8935689,8935689,14366,50,1857231384,9.781857e+12,Iain M. Banks,1987.0,Consider Phlebas,"Consider Phlebas (Culture, #1)",eng,3.85,48649,53499,2666,1083,3656,12772,20533,15455,https://images.gr-assets.com/books/1327951890m...,https://images.gr-assets.com/books/1327951890s...
1823,1992,b1993,31332,31332,2925979,63,345434803,9.780345e+12,Anne Rice,1998.0,The Vampire Armand,"The Vampire Armand (The Vampire Chronicles, #6)",en-US,3.75,54919,57566,756,1129,4866,17158,18670,15743,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
1824,1996,b1997,9565548,9565548,14452295,33,054762834X,9.780548e+12,Robin LaFevers,2012.0,Grave Mercy,"Grave Mercy (His Fair Assassin, #1)",eng,3.92,70476,74683,7661,2257,4487,15591,27223,25125,https://images.gr-assets.com/books/1320269319m...,https://images.gr-assets.com/books/1320269319s...


In [7]:
ratings_df

,Unnamed: 0,user_id,book_id,rating
0,0,u1,b258,5
1,130,u11,b258,3
2,1998,u143,b258,4
3,4731,u325,b258,4
4,5510,u362,b258,2
...,...,...,...,...
124757,438735,u1920,b1499,4
124758,439348,u334,b1499,2
124759,439505,u927,b1499,4
124760,439728,u1298,b1499,4


In [8]:
to_read_df

,Unnamed: 0,user_id,book_id
0,386014,u1333,b1797
1,36194,u765,b1797
2,25568,u1042,b1797
3,596738,u1360,b1797
4,52814,u1782,b1797
...,...,...,...
135610,444981,u556,b1157
135611,428922,u343,b1157
135612,457472,u1193,b1743
135613,487831,u1121,b1743


In [9]:
test

,Unnamed: 0,user_id,book_id
0,26204,u978,b323
1,74616,u109,b323
2,334583,u1721,b323
3,297262,u429,b323
4,30956,u339,b323
...,...,...,...
33912,347299,u272,b1545
33913,1881,u126,b1333
33914,400826,u1841,b1513
33915,518727,u1599,b1513


Here we construct the Interactions objects from `ratings.csv`, `to_read.csv`, and `test.csv`. We manually specify the `num_users` and `num_items` parameters to all Interactions objects, in case the test set differs from your training sets.

In [10]:
# Do NOT change the order of generating uids.
uid_map = defaultdict(count().__next__)
test_uids = np.array([uid_map[uid] for uid in test['user_id'].values], dtype = np.int32)
rating_uids = np.array([uid_map[uid] for uid in ratings_df['user_id'].values], dtype = np.int32)
toread_uids = np.array([uid_map[iid] for iid in to_read_df['user_id'].values], dtype = np.int32)

# Do NOT change the order of generating iids.
iid_map = defaultdict(count().__next__)
rating_iids = np.array([iid_map[iid] for iid in ratings_df['book_id'].values], dtype = np.int32)
test_iids = np.array([iid_map[iid] for iid in test['book_id'].values], dtype = np.int32)
toread_iids = np.array([iid_map[iid] for iid in to_read_df['book_id'].values], dtype = np.int32)

uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}

rating_dataset = Interactions(
    user_ids = rating_uids,
    item_ids = rating_iids,
    ratings = ratings_df['rating'].values,
    num_users = len(uid_rev_map),
    num_items = len(iid_rev_map)
)

toread_dataset = Interactions(
    user_ids = toread_uids,
    item_ids = toread_iids,
    num_users = len(uid_rev_map),
    num_items = len(iid_rev_map)
)

test_dataset = Interactions(
    user_ids = test_uids,
    item_ids = test_iids,
    num_users = len(uid_rev_map),
    num_items = len(iid_rev_map)
)

print(rating_dataset)
print(toread_dataset)
print(test_dataset)

toread_dataset_train, validation = random_train_test_split(toread_dataset, random_state = np.random.RandomState(SEED))  # Here we define the validation set.
print(toread_dataset_train)
print(validation)

<Interactions dataset (1999 users x 1826 items x 124762 interactions)>
<Interactions dataset (1999 users x 1826 items x 135615 interactions)>
<Interactions dataset (1999 users x 1826 items x 33917 interactions)>
<Interactions dataset (1999 users x 1826 items x 108492 interactions)>
<Interactions dataset (1999 users x 1826 items x 27123 interactions)>


Finally, this is some utility code that we will use in the exercise.

In [11]:
def get_author_title(iid: int) -> str:
  '''
  Get the author and title of a specific book.

  Parameters
  ----------
  iid : the iid of the book

  Returns
  -------
  author_title : the author and title of a specific book in the format of "<authors> / <title>"
  '''

  book_id = iid_rev_map[iid]
  row = books_df[books_df['book_id'] == book_id]
  return row.iloc[0]['authors'] + ' / ' + row.iloc[0]['title']


print('iid 0:', get_author_title(0))

iid 0: Carlos Ruiz Zafón, Lucia Graves / The Shadow of the Wind (The Cemetery of Forgotten Books,  #1)


## Pre 3. Example Code

To evaluate some of your hand-implemented recommender systems (e.g. Task 1, Task 4), you will need to instantiate objects that match the specification of a Spotlight model, which `mrr_score()` etc. expects.

Here is an example recommender object that returns 0 for each item, regardless of user.

In [12]:
class DummyModel:
  '''
  The class for a dummy model as an example.
  '''

  def __init__(self, n_items: int) -> None:
    '''
    The constructor of the class for a dummy model as an example.

    Parameters
    ----------
    n_items : the number of items
    '''

    self.predictions = np.zeros(n_items)

  def predict(self, uid: int) -> np.ndarray:
    '''
    Make predictions regardless of the user.

    Parameters
    ----------
    uid : the user we are requesting recommendations for

    Returns
    -------
    predictions : 0 scores, one for each item, same for any user
    '''

    return self.predictions


print(mrr_score(
    DummyModel(test_dataset.num_items),
    test_dataset,
    train = rating_dataset,
    k = 100,
    verbose = True  # Display a progress bar.
).mean())

1999it [00:00, 2564.79it/s]

0.0


# Part-A. Combination of Recommendation Models

## Task 1. Explicit & Implicit Matrix Factorisation Models

Create and train three matrix factorisation systems:
- EMF: explicit MF, trained on the `ratings` Interactions object (`rating_dataset`)
- IMF: implicit MF, trained on the `toread` Interactions object (`toread_dataset_train`)
- BPRMF: implicit MF with the BPR loss function (`loss = 'bpr'`), trained on the `toread` Interactions object (`toread_dataset_train`)

Use a variable of the same name for these models, as we will use some of them later (e.g. `BPRMF`). In all cases, you must use the standard initialisation arguments except `random_state = np.random.RandomState(SEED)`. Evaluate each of these models in terms of mean reciprocal rank (MRR) on the test set. MRR can be obtained using:
```python
mrr_score(X, test_dataset, train = rating_dataset, k = 100, verbose = True).mean()
```
where X is an instance of a Spotlight model. Do NOT change the `k` or `train` arguments.

In [13]:
# Train and evaluate EMF.
EMF = ExplicitFactorizationModel(random_state = np.random.RandomState(SEED))  # Keep default parameters except random_state.
EMF.fit(rating_dataset, verbose = True)
mrr_score(EMF, test_dataset, train = rating_dataset, k = 100).mean()

Epoch 0: loss 3.8710271667261593
Epoch 1: loss 0.7940810446123607
Epoch 2: loss 0.6382512643200452
Epoch 3: loss 0.5217335281557725
Epoch 4: loss 0.44844855655167926
Epoch 5: loss 0.40543351120880394
Epoch 6: loss 0.3823863151254224
Epoch 7: loss 0.36336620252762664
Epoch 8: loss 0.35137936695799477
Epoch 9: loss 0.3396692546237199


0.05898399982013507

In [14]:
# Train and evaluate IMF.
IMF = ImplicitFactorizationModel(random_state = np.random.RandomState(SEED))  # Keep default parameters except random_state.
IMF.fit(toread_dataset_train, verbose = True)
mrr_score(IMF, test_dataset, train = rating_dataset, k = 100).mean()

Epoch 0: loss 0.7677980539090229
Epoch 1: loss 0.53877861825925
Epoch 2: loss 0.47017199658560305
Epoch 3: loss 0.428322009882837
Epoch 4: loss 0.39839018825090156
Epoch 5: loss 0.368275504770144
Epoch 6: loss 0.3473479778699155
Epoch 7: loss 0.32980164804689166
Epoch 8: loss 0.31870100696413023
Epoch 9: loss 0.3048194432103971


0.3299315791285401

In [15]:
# Train and evaluate BPRMF.
BPRMF = ImplicitFactorizationModel(loss = 'bpr', random_state = np.random.RandomState(SEED))  # Keep default parameters except loss and random_state.
BPRMF.fit(toread_dataset_train, verbose = True)
rrs_BPRMF = mrr_score(BPRMF, test_dataset, train = rating_dataset, k = 100)  # Store each user's RR for later use.
rrs_BPRMF.mean()

Epoch 0: loss 0.33895447579616644
Epoch 1: loss 0.19644999289709442
Epoch 2: loss 0.15870640168563938
Epoch 3: loss 0.14147728193059284
Epoch 4: loss 0.132827276100387
Epoch 5: loss 0.12213623321632731
Epoch 6: loss 0.11668406535853755
Epoch 7: loss 0.11047121562626001
Epoch 8: loss 0.10888675406996934
Epoch 9: loss 0.10400472129782978


0.4076771464674879

## Task 2. Hybrid Model

In this task, you are expected to create new hybrid recommendation models that 
combine the two models in Task 1, namely IMF and BPRMF. 

(a) Linearly combine the *scores* from IMF and BPRMF using the data fusion CombSUM. Normalise both input scores into the range [0,1] using sklearn's [`minmax_scale()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html) function before combining them.

(b) Apply a pipelining recommender, where the top 100 items are obtained from IMF and re-ranked using the scores of BPRMF. Items not returned by IMF get a score of 0.

To implement these hybrid models, you should create new classes that abide by the Spotlight model contract (namely, it has a `predict(self, uid)` function that returns a score for *all* items). Evaluate each model in terms of MRR. How many users are improved, how many are degraded compared to the BPRMF baseline?

Finally, pass your instantiated model object to the `test_Hybrid_a()` (for (a)) or `test_Hybrid_b()` (for (b)) functions, as appropriate, and record the results in the quiz. For example, if your model for (b) is called `pipeline`, then you would run:
```python
test_Hybrid_b(pipeline)
```

You now have sufficient information to answer the Task 2 quiz questions.

In [16]:
def test_Hybrid_a(hybrid_a) -> None:
  '''
  Test Hybrid A.

  Parameters
  ----------
  hybrid_a : an instantiation of the model Hybrid A
  '''

  for i, uid in enumerate([5, 20]):
    print('Hybrid A test case %d (uid %d):' % (i, uid))
    print(np.count_nonzero(hybrid_a.predict(uid) > 1))

def test_Hybrid_b(hybrid_b) -> None:
  '''
  Test Hybrid B.

  Parameters
  ----------
  hybrid_b : an instantiation of the model Hybrid B
  '''

  for i, iid in enumerate([3, 0]):
    print('Hybrid B test case %d (iid %d):' % (i, iid))
    print(hybrid_b.predict(0)[iid])

In [17]:
class HybridA:
  '''
  The class for the model Hybrid A.
  '''

  def __init__(self, IMF: ImplicitFactorizationModel, BPRMF: ImplicitFactorizationModel) -> None:
    '''
    The constructor of the class for the model Hybrid A.

    Parameters
    ----------
    IMF : the trained model IMF
    BPRMF : the trained model BPRMF
    '''

    self.IMF = IMF
    self.BPRMF = BPRMF

  def predict(self, uid: int) -> np.ndarray:
    '''
    Make predictions.

    Parameters
    ----------
    uid : the user we are requesting recommendations for

    Returns
    -------
    predictions : the predictions for each item for a specific user.
    '''
    
    return minmax_scale(self.IMF.predict(uid)) + minmax_scale(self.BPRMF.predict(uid))  # Normalise predictions, and apply (parallelised) weighted hybridisation using CombSUM as the data fusion.

class HybridB:
  '''
  The class for the model Hybrid B.
  '''

  def __init__(self, IMF: ImplicitFactorizationModel, BPRMF: ImplicitFactorizationModel) -> None:
    '''
    The constructor of the class for the model Hybrid B.

    Parameters
    ----------
    IMF : the trained model IMF
    BPRMF : the trained model BPRMF
    '''

    self.IMF = IMF
    self.BPRMF = BPRMF

  def predict(self, uid: int) -> np.ndarray:
    '''
    Make predictions.

    Parameters
    ----------
    uid : the user we are requesting recommendations for

    Returns
    -------
    predictions : the predictions for each item for a specific user.
    '''

    IMF_ranks = np.argsort(-self.IMF.predict(uid))[:100]  # The top 100 items obtained from IMF.
    BPRMF_predictions = self.BPRMF.predict(uid)
    predictions = np.zeros_like(BPRMF_predictions, dtype = BPRMF_predictions.dtype)  # Generate an initial prediction ndarray with 0.
    predictions[IMF_ranks] = BPRMF_predictions[IMF_ranks]  # Assign the top 100 items to BPRMF's scores.
    return predictions  # Return predictions applying (pipelined) cascade hybridisation.

In [18]:
hybrid_a = HybridA(IMF, BPRMF)
rrs_a = mrr_score(hybrid_a, test_dataset, train = rating_dataset, k = 100)
print('The number of improved users:', np.where(rrs_BPRMF < rrs_a)[0].size)
print('The number of downgraded users:', np.where(rrs_BPRMF > rrs_a)[0].size)
print('MRR:', rrs_a.mean())

The number of improved users: 736
The number of downgraded users: 745
MRR: 0.41010939818893816


In [19]:
hybrid_b = HybridB(IMF, BPRMF)
rrs_b = mrr_score(hybrid_b, test_dataset, train = rating_dataset, k = 100)
print('The number of improved users:', np.where(rrs_BPRMF < rrs_b)[0].size)
print('The number of downgraded users:', np.where(rrs_BPRMF > rrs_b)[0].size)
print('MRR:', rrs_b.mean())

The number of improved users: 586
The number of downgraded users: 214
MRR: 0.41511955311399246


In [20]:
test_Hybrid_a(hybrid_a)
test_Hybrid_b(hybrid_b)

Hybrid A test case 0 (uid 5):
445
Hybrid A test case 1 (uid 20):
407
Hybrid B test case 0 (iid 3):
22.013418
Hybrid B test case 1 (iid 0):
0.0


# Part-B. Analysing Recommendation Models

## Utility methods

Below, we provide a function, `get_top_K()` which, when provided with a Spotlight model, will provide the top `k` predictions for the specified `uid`. The `iids`, their scores, and their embeddings are returned. 

In [21]:
def get_top_K(model, uid: int, k: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  '''
  
  Parameters
  ----------
  model : the trained model
  uid : the uid of the specific user
  k : the number of top predictions kept

  Returns
  -------
  top_k : the iids, their normalised scores in descending order, and item embeddings for the top k predictions of the specific user
  '''

  scores = model.predict(uid)  # Get scores from the model.
  scores = minmax_scale(scores)  # Normalise scores.
  ranks = rankdata(-scores)  # Compute their ranks.
  
  # Get and filter iids, scores, and embeddings.
  rtr_scores = scores[ranks <= k]
  rtr_iids = np.argwhere(ranks <= k).flatten()

  if hasattr(model, '_net'):
    embs = model._net.item_embeddings.weight[rtr_iids]
  # The model does not have any embeddings.
  else:  
    embs = np.zeros([k, 1])
  
  ordering = np.argsort(-rtr_scores)  # Identify correct ordering.
  return rtr_iids[ordering], rtr_scores[ordering], embs[ordering]


if BPRMF is not None:
  iids, scores, embs = get_top_K(BPRMF, 0, 10)
  print('Returned iids:', iids)
  print('Returned scores:', scores)
  print('Returned embeddings:', embs)
else:
  print('You need to define BPRMF in Task 1.')

Returned iids: [ 23 108  21  33   9  81  52 254  16   3]
Returned scores: [1.         0.9895131  0.9848315  0.92250896 0.9070817  0.90654314
 0.9005319  0.89310133 0.88378096 0.8836929 ]
Returned embeddings: tensor([[-0.0453,  1.3716, -0.8307, -1.2616,  1.6700,  1.0161,  1.1168,  2.3530,
         -1.2027,  0.8522, -1.0941, -0.6865, -0.5725, -2.0335, -1.2591,  0.6154,
         -0.1374, -1.6868, -1.8615, -0.7514,  1.9909, -0.3909,  1.9239,  1.3293,
         -1.2834, -0.4520,  1.1338,  0.3467,  2.5169, -2.1587,  1.2310,  1.1670],
        [ 0.1239,  1.1004,  0.0531, -1.1045,  1.9932,  1.5049,  1.0011,  1.9734,
         -1.6322, -0.8913, -0.6372,  0.7721, -1.1422, -2.2424, -1.1936, -0.5770,
          0.0762, -1.0283, -1.2807, -2.0889,  2.8154, -0.9600, -0.1419,  0.8408,
         -1.6067, -1.2905,  1.9169,  1.3988,  1.8646, -2.2028,  0.5365,  0.2022],
        [ 0.3845,  0.8188, -0.1892, -1.1793,  2.1731,  0.6669,  1.1271,  1.4538,
         -1.2173, -0.5447, -1.6713,  0.5249, -0.6132, -3.1082

## Task 3. Evaluation of Non-personalised Models
Implement the following four (non-personalised) baselines for ranking books based on their statistics:
- Average rating, obtained from `ratings_df` (column `ratings`)
- Number of ratings, obtained from `books_df` (column `ratings_count`)
- Number of 5* ratings, obtained from `books_df` (column `ratings_5`)
- Fraction of 5* ratings, calculated from the two sources of evidence above (columns `ratings_5` and `ratings_count`).

Evaluate these in terms of MRR using the provided test data. You may use the `StaticModel` class below. 

**Hints:**
- As in Exercise 2, the order of items returned by `predict()` is *critical*. You may wish to refer to `iid_map`.
- For all models, you need to ensure that your values are not cast to ints. If you are extracting values from a Pandas series, it is advised to use [`astype(np.float32)`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.astype.html).


In [22]:
class StaticModel:
  '''
  The class for a baseline static scoring model.
  '''

  def __init__(self, static_scores: np.ndarray) -> None:
    '''
    The constructor of the class for a baseline static scoring model.

    Parameters
    ----------
    static_scores : the static scores
    '''

    assert isinstance(static_scores, np.ndarray), 'Expected a numpy array.'
    assert static_scores.dtype == np.float32 or static_scores.dtype == np.float64, 'Expected a numpy array of floats.'
    self.static_scores = static_scores
  
  def predict(self, uid: int) -> np.ndarray:
    '''
    Predict the static scores regardless of the user.

    Parameters
    ----------
    uid : the user we are requesting recommendations for
    
    Returns
    -------
    scores : the static scores, one for each item, same for any user
    '''
    
    return self.static_scores

In [23]:
# The assumed item order in StaticModel's predict() is the same as iid_map (i.e., the order of unique books in ratings_df).
n_ratings = np.array([books_df[books_df['book_id'] == id]['ratings_count'].values.astype(np.float32) for id in iid_map.keys()])
n_ratings_5 = np.array([books_df[books_df['book_id'] == id]['ratings_5'].values.astype(np.float32) for id in iid_map.keys()])

model_a = StaticModel(ratings_df.groupby(['book_id'], sort = False)['rating'].mean().values)  # A static model based on the average rating. Use "sort = False" to preserve the order of unique books in ratings_df.
model_b = StaticModel(n_ratings)  # A static model based on the number of ratings.
model_c = StaticModel(n_ratings_5)  # A static model based on the number of 5* ratings.
model_d = StaticModel(n_ratings_5 / n_ratings)  # A static model based on the fraction of 5* ratings.

print('MRR of Model A:', mrr_score(model_a, test_dataset, train = rating_dataset, k = 100).mean())
print('MRR of Model B:', mrr_score(model_b, test_dataset, train = rating_dataset, k = 100).mean())
print('MRR of Model C:', mrr_score(model_c, test_dataset, train = rating_dataset, k = 100).mean())
print('MRR of Model D:', mrr_score(model_d, test_dataset, train = rating_dataset, k = 100).mean())

MRR of Model A: 0.015052024168984034
MRR of Model B: 0.2396001188245477
MRR of Model C: 0.2409670879930144
MRR of Model D: 0.03415267465103555


## Task 4. Qualitatively Examining Recommendations

From now on, we will consider the `BPRMF` model.

In Recommender Systems, the ground truth (i.e. our list of books that the user has added to their "to-read" shelf) can be very incomplete. For instance, this can be because the user is not aware of the book yet.

For this reason, it is important to "eyeball" the recommendations, to understand what the system is surfacing, and whether the recommendations make sense. In this way, we understand if the recommendations are reasonable, even if they are for books that the user has not actually read according to the test dataset.

First, write a function, which given a `uid` (int), prints the *title and authors* of:
- (a) the books that the user has previously shelved (c.f. `toread_dataset`)
- (b) the books that the user will read in the future (c.f. `test_dataset`)
- (c) the top 10 books that the user were recommended by `BPRMF` - you can make use of `get_top_K()`.

You can use the previously defined `get_author_title()` function in your solution. You will also want to compare books in (c) with those in (a) and (b).

In [24]:
def list_books(uid: int) -> None:
  '''
  List the books that the a specific user has previously shelved, or will read in the future, or the top 10 books that the user were recommended by BPRMF.

  Parameters
  ----------
  uid : the uid of the specific user
  '''

  print('Books that the user has previously shelved:')

  for iid in toread_dataset.item_ids[toread_dataset.user_ids == uid]:
    print('\t' + get_author_title(iid))
  
  print('\nBooks that the user will read in the future:')

  for iid in test_dataset.item_ids[test_dataset.user_ids == uid]:
    print('\t' + get_author_title(iid))

  print('\nTop 10 books that the user were recommended by BPRMF:')
  top_k_iids, _, _ = get_top_K(BPRMF, uid, 10)

  for iid in top_k_iids:
    print('\t' + get_author_title(iid))
  
  top_k_ids = [iid_rev_map[iid] for iid in top_k_iids]
  top_k_authors = books_df[np.isin(books_df['book_id'], top_k_ids)]['authors'].values
  test_ids = [iid_rev_map[iid] for iid in test_dataset.item_ids[test_dataset.user_ids == uid]]
  test_authors = books_df[np.isin(books_df['book_id'], test_ids)]['authors'].values
  print('\nSummary:')
  print('\tRR:', rrs_BPRMF[uid])
  print('\tThe number of the top 10 books that were already previously shelved:', np.count_nonzero(np.isin(top_k_iids, toread_dataset.item_ids[toread_dataset.user_ids == uid])))
  print('\tThe number of the top 10 books that were also present in the future shelf:', np.count_nonzero(np.isin(top_k_iids, test_dataset.item_ids[test_dataset.user_ids == uid])))
  print('\tThe number of the top 10 books\' authors that were also present in the future shelf:', np.count_nonzero(np.isin(top_k_authors, test_authors)))

Then, we will examine two specific users, namely `uid` 1805 (u336) and `uid` 179 (user u1331), to analyse if their recommendations make sense. The results could also be referred to in the following tasks.

In [25]:
list_books(1805)

Books that the user has previously shelved:
	Stieg Larsson, Reg Keeland / The Girl Who Kicked the Hornet's Nest (Millennium, #3)
	Suzanne Collins / Mockingjay (The Hunger Games, #3)
	Dennis Lehane / Shutter Island
	Suzanne Collins / Catching Fire (The Hunger Games, #2)
	Paula Hawkins / The Girl on the Train
	Robert Ludlum / The Bourne Supremacy (Jason Bourne, #2)
	John Grisham / The Client
	Thomas Harris / The Silence of the Lambs  (Hannibal Lecter, #2)
	Daphne du Maurier, Sally Beauman / Rebecca
	Robert Ludlum / The Bourne Identity (Jason Bourne, #1)
	Robert Galbraith, J.K. Rowling / The Cuckoo's Calling (Cormoran Strike, #1)
	Stephen King / Misery
	Michael Crichton / Jurassic Park (Jurassic Park, #1)
	Robert Ludlum / The Bourne Ultimatum (Jason Bourne, #3)
	Stephen King, Bernie Wrightson / The Stand
	Michael Crichton / The Andromeda Strain
	Thomas Harris / Red Dragon (Hannibal Lecter, #1)
	Lee Child / Die Trying (Jack Reacher, #2)
	Lee Child / Worth Dying For (Jack Reacher, #15)
	Lee

In [26]:
list_books(179)

Books that the user has previously shelved:
	Dan Brown / Angels & Demons  (Robert Langdon, #1)
	Suzanne Collins / The Hunger Games (The Hunger Games, #1)
	Antoine de Saint-Exupéry, Richard Howard, Dom Marcos Barbosa, Melina Karakosta / The Little Prince
	Truman Capote / Breakfast at Tiffany's
	Dan Brown / The Da Vinci Code (Robert Langdon, #2)
	Laura Ingalls Wilder, Garth Williams / Little House on the Prairie (Little House, #2)
	Milan Kundera, Michael Henry Heim / The Unbearable Lightness of Being
	Suzanne Collins / Catching Fire (The Hunger Games, #2)
	John Grisham / The Client
	J.R.R. Tolkien / The Lord of the Rings (The Lord of the Rings, #1-3)
	J.R.R. Tolkien / The Hobbit
	Margaret Mitchell / Gone with the Wind
	Neil Gaiman / Stardust
	Laura Ingalls Wilder, Garth Williams / Little House in the Big Woods (Little House, #1)
	Pearl S. Buck / The Good Earth (House of Earth, #1)
	Dan Brown / Digital Fortress
	Daniel Keyes / Flowers for Algernon
	Neil Gaiman / Coraline
	Dan Brown / Dece

# Part-C. Diversity of Recommendations

This part of the exercise is concerned with diversification, as covered in Lecture 11.

## Task 5. Measuring Intra-List Diversity (ILD)

For the BPR implicit factorisation model, implement the intra-list diversity measure (see Lecture 11) of the top 5 scored items based on their item embeddings in the `BPRMF` model. Implement your ILD as a function with the specification:
```python
def measure_ild(top_books: np.ndarray, K: int = 5) -> float
```
where:
- `top_books` is a Numpy array of `iids` that have been returned for a particular user. For instance, it can be obtained from `get_top_K()`.
- `K` is the number of top-ranked items to consider from `top_books`.
- Your implementation should use the item emebddings stored in the `BPRMF` model.

Calculate the ILD (with `K` = 5). Using your code for Task 4, identify the books previously shelved and recommended for the specific users requested in the quiz, and use these to analyse the recommendations.

**Hints:**
- As can be seen in `get_top_K()`, item embeddings can be obtained from `BPRMF._net.item_embeddings.weight[iid]`.
- For obtaining the cosine similarity of PyTorch tensors, use `nn.functional.cosine_similarity(x, y, dim = 0)`.

In [27]:
def measure_ild(top_books: np.ndarray, K: int = 5) -> float:
  '''
  Measure the ILD.

  Parameters
  ----------
  top_books : the iids that have been returned for a particular user
  K : the number of top-ranked items to consider from top_books

  Returns
  -------
  ild : the ILD
  '''

  embeddings = BPRMF._net.item_embeddings.weight[top_books[:K]]  # Get corresponding K item embeddings.
  cos = np.array([nn.functional.cosine_similarity(embeddings[i], embeddings[j], dim = 0).item() for i in range(K) for j in range(i + 1, K)])  # Compute cosine similarities for unique pairs.
  return np.sum(1 - cos) / (K * (K - 1) / 2)  # ILD = SUM(1 - Similarities) / (n * (n - 1) / 2)

In [28]:
top_k_iids, _, _ = get_top_K(BPRMF, 1805, 5)
measure_ild(top_k_iids)

0.7484898872673511

In [29]:
top_k_iids, _, _ = get_top_K(BPRMF, 179, 5)
measure_ild(top_k_iids)

0.27872883677482607

## Task 6. Implement MMR Diversification

Develop an Maximal Marginal Relevance (M**M**R) diversification technique, to re-rank the top-ranked recommendations for a given user. Your function should adhere to the specification as follows:
```python
def mmr(iids: Sequence[int], scores: Sequence[float], embs: np.ndarray, alpha: float) -> np.ndarray:
```
where `iids` is a list of iids, `scores` is their corresponding scores (in descending order), `embs` is their embeddings, and `alpha` controls the diversification trade-off. The function returns a re-ordering of `iids`. As in previous Exercises, type hints are provided for clarity.

**Hints:**
- As above, for obtaining the cosine similarity of PyTorch tensors, use `nn.functional.cosine_similarity(x, y, dim = 0)`.

To use your `mmr()` function, provide it with the outputs of `get_top_K()`. For example, to obtain an MMR reordering of the top 10 predictions of `uid` 0, the following code could help:
```
mmr(*get_top_K(BPRMF, 0, 10), 0.5)
```

Thereafter, we provide test cases for your MMR implementation, which you  should report in the quiz. We also ask for the ILD values before and after the application of MMR.

In [30]:
def mmr(iids: Sequence[int], scores: Sequence[float], embs: np.ndarray, alpha: float) -> np.ndarray:
  '''
  Re-rank the top-ranked recommendations for a given user based on MMR.

  Parameters
  ----------
  iids : the iids for the top-ranked predictions of the specific user
  scores : the normalised scores in descending order for the top-ranked predictions of the specific user
  embs : the item embeddings for the top-ranked predictions of the specific user
  alpha : the parameter for controlling the diversification trade-off in computing the MMR

  Returns
  -------
  rtr_iids : a re-ordering of iids
  '''

  assert len(iids) == len(scores), 'Expected equal size of iids and scores.'
  assert len(iids) == embs.shape[0], 'Expected equal size of iids and item embeddings.'
  assert len(embs.size()) == 2, 'Expected correct shape of item embeddings.'
  rank = []  # A rank list with indices sorted by MMR.

  # Ensure a list is converted to an ndarray.
  iids = np.array(iids)
  scores = np.array(scores)

  # Iterate for each iid. MMR = ARGMAX(Relevance - Novelty) = ARGMAX(alpha * Score - (1 - alpha) * Similarity)
  for _ in range(len(iids)):
    mmrs = []
    waiting = np.delete(range(len(iids)), rank)  # The iids pending ranking.
    
    # Iterate for each iid NOT in rank.
    for i in waiting:
      novelties = [nn.functional.cosine_similarity(embs[i], embs[j], dim = 0).item() for j in rank]  # Iterate for each iid in rank.
      mmrs.append(alpha * scores[i] - (0 if len(rank) == 0 else (1 - alpha) * np.max(novelties)))  # In Iteration 1, no element in rank, novelty equals 0, so the 1st rank is Index 0 of scores which is in descending order.

    rank.append(waiting[np.argmax(mmrs)])  # Add the index of the iid with the maximum MMR in this iteration.

  return iids[np.array(rank)]

In [31]:
def run_MMR_testcases(mmr_fn) -> None:
  '''
  Run several MMR testcases.

  Parameters
  ----------
  mmr_fn : the function (a first-class object) that re-ranks the top-ranked recommendations for a given user based on MMR.
  '''

  example_embeddings1 = torch.tensor([[1.0, 1.0], [1.0, 1.0], [0, 1.0], [0.1, 1.0]])
  example_embeddings2 = torch.tensor([[1.0, 1.0], [1.0, 1.0], [0.02, 1.0], [0.01, 1.0]])
  print('Testcase 0:', mmr_fn([1, 2, 3, 4], [0.5, 0.5, 0.5, 0.5], example_embeddings1, 0.5)[0])
  print('Testcase 1:', mmr_fn([1, 2, 3, 4], [0.5, 0.5, 0.5, 0.5], example_embeddings1, 0.5)[1])
  print('Testcase 2:', mmr_fn([1, 2, 3, 4], [4, 3, 2, 1], example_embeddings1, 1)[1])
  print('Testcase 3:', mmr_fn([1, 2, 3, 4], [0.99, 0.98, 0.97, 0.001], example_embeddings2, 0.001)[1])
  print('Testcase 4:', mmr_fn([1, 2, 3, 4], [0.99, 0.98, 0.97, 0.001], example_embeddings2, 0.5)[1])

run_MMR_testcases(mmr)

Testcase 0: 1
Testcase 1: 3
Testcase 2: 2
Testcase 3: 4
Testcase 4: 3


Now we can analyse the impact of our MMR implementation. Let's consider again `uid` 179 (user u1331). Apply MMR on the top 10 results obtained from the BPRMF model using `get_top_K()`, with an alpha value of 0.5.

Finally, anaylse the returned books. Calculate the ILD (with `k = 5`), and examine the authors and titles (using `get_author_title()`). 

Now answer the questions in Task 6 of the Moodle quiz.


In [32]:
iids, scores, embeddings = get_top_K(BPRMF, 179, 10)
rtr_iids = mmr(iids, scores, embeddings, 0.5)
print('ILD:', measure_ild(rtr_iids))
print('Books (sorted by MMR):')

for rtr_iid in rtr_iids:
  print('\t' + get_author_title(rtr_iid))

print('Corresponding iids:', rtr_iids)

ILD: 0.5566449973732233
Books (sorted by MMR):
	John Grisham / The Partner
	J.K. Rowling, Mary GrandPré / Harry Potter and the Sorcerer's Stone (Harry Potter, #1)
	John Grisham / The Street Lawyer
	John Grisham / The Pelican Brief
	John Grisham / The Brethren
	John Grisham / The Rainmaker
	John Grisham / The Runaway Jury
	John Grisham / The Broker
	John Grisham / The Client
	John Grisham / The King of Torts
Corresponding iids: [ 89   9  88 391 906  62 934  86  92  91]


# Task 7

This task is not a practical task - instead there are questions that tests your understanding of some related content of the course in the quiz.

# End of Exercise

As part of your submission, you should complete the Exercise 3 quiz on Moodle.
You will need to upload your notebook, complete with the **results** of executing the code.